In [1]:
"""
2 stage ensemble for object detection models.

Without processing power to run all 4 models at the same time, so we propose an alternative:
we run 4 models on the same test data separately, gather the output and ensemble.

Stage 1:
- For each test image:
- Run inference using our 4 models, store outputs.
- Convert outputs to EB format, 
    by calling read_RODM_pred() for each model output.
    Append fn outputs to master lists.
- Apply ensemble (concensus/majority voting) on master lists.
- Store ensemble output to new txt file in RODM format.
- Show ensemble pictures.
- Run RODM on ensemble output to calculate mAP.

Stage 2:
- For each test image:
- Run inference using stock YOLOv4 trained on COCO, store outputs
    use 'Get YOLO4 output.ipynb' to convert YOLOv4 output to RODM format.
- Gather ensemble output from Stage 1, and YOLOv4 converted output
- Apply ensemble (affirmative/or method) function in EB.
- Store stage 2 ensemble output to new txt files.
- Show stage 2 ensemble pictures.
- No need to run RODM and calculate mAP. It's impossible because test data isn't annotated for 80 COCO classes.

Some Notes:
- Enselbme Boxes input format: [x1, y1, x2, y2] normalized.
- Review Object Detection Metrics input format: [class_id, confidence, x, y, w, h] where coords are normalized.
- NMW non maximum weighted is a different NMS, where for each detection all boxes are considered, and a new weighted average
    box is created and original boxes discarded.
- Test dataset should have 1497 images, excluding B111.jpg, or B111.txt for predictions.
    

Input Directories:
- All folders below are located in D:\DL_data\Accuracy\
- Custom YOLOv4 predictions: YOLO_test_output
- Custom CenterNet predictions: CenterNet_predictions_100bboxes  (100 bboxes, no NMS, no confidence filter)
  - after NMS and confidence filter: CenterNet_nms_conf
- Custom EfficientDet predicitions: Eff_test_results
- Custom SSD predictions: SSD_predictions_100bboxes  (100 bboxes, no NMS, no confidence filter)
  - after NMS and confidence filter: SSD_nms_conf
- Stock YOLOv4 predictions:

"""

"\n2 stage ensemble for object detection models.\n\nStage 1:\n- For each test image:\n- Run inference using our 4 models, store outputs.\n- Convert outputs to EB format, \n    by calling read_RODM_pred() for each model output.\n    Append fn outputs to master lists.\n- Run weighted boxes fusion (concensus/majority voting) on master lists.\n- Store ensemble output to new txt file in RODM format.\n- Show ensemble pictures.\n- Run RODM on ensemble output to calculate mAP.\n\nStage 2:\n- For each test image:\n- Run inference using stock YOLOv4 trained on COCO, store outputs\n    use 'Get YOLO4 output.ipynb' to convert YOLOv4 output to RODM format.\n- Gather ensemble output from Stage 1, and YOLOv4 converted output\n- Apply ensemble (affirmative/or method) using NMS function in EB.\n- Store stage 2 ensemble output to new txt files.\n- Show stage 2 ensemble pictures.\n- No need to run RODM and calculate mAP. It's impossible because test data isn't annotated for 80 COCO classes.\n\nSome Notes

### Functions

In [16]:
import cv2
import random
import os
import numpy as np
from ensemble_boxes import *

#### show image

In [97]:
def show_image(im, name='image'):
    resized = ResizeWithAspectRatio(im, height=900)    # Can resize by width or height
    cv2.imshow(name, resized)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [92]:
# to resize cv2 imshow window
def ResizeWithAspectRatio(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    return cv2.resize(image, dim, interpolation=inter)

#### show boxes

In [3]:
def class_colors(names):
    """
    Create a dict with one random BGR color for each
    class name
    """
    return {name: (
        random.randint(0, 255),
        random.randint(0, 255),
        random.randint(0, 255)) for name in names}

In [4]:
def show_boxes(boxes_list, scores_list, labels_list, image_path):  
    
    # to correctly call color fn, unpack labels_list
    labels_unpacked =[]
    for lst in labels_list:
        labels_unpacked.extend(lst)
        
    color_list = class_colors(labels_unpacked)    # generate random color for each predicted class
    
    image = cv2.imread(image_path)    # load image file
    height, width = image.shape[:2]    # get image size
    thickness = 2    # thickness of bbox border
    
    for i in range(len(boxes_list)):    # loop thru each predicted image?
        for j in range(len(boxes_list[i])):    # loop thru each bbox for each image?
            x1 = int(width * boxes_list[i][j][0])
            y1 = int(height * boxes_list[i][j][1])
            x2 = int(width * boxes_list[i][j][2])
            y2 = int(height * boxes_list[i][j][3])
            lbl = labels_list[i][j]    # this is the predicted class value in int
            
            cv2.rectangle(image, (x1, y1), (x2, y2), color_list[lbl], thickness)    # now we can call color_list[class #]
            cv2.putText(image, '{} [{:.2f}]'.format(label_names[lbl]['name'], float(scores_list[i][j])),
                       (x1, y2-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                       color_list[lbl], 1)
    show_image(image)
    
    #### if need to resize cv2 show window size, see 9. 32 classes scratch.ipynb

In [89]:
# hard coded here, so that we don't need to read any file to get class names
# notice this is a list of dictionaries
label_names = [ # 23 road mark classes
          {'name':'25', 'id':1}, {'name':'30', 'id':2}, 
          {'name':'35', 'id':3}, {'name':'40', 'id':4},
          {'name':'45', 'id':5}, {'name':'50', 'id':6},
          {'name':'bike', 'id':7}, {'name':'bus', 'id':8},
          {'name':'diamond', 'id':9}, {'name':'F', 'id':10},
          {'name':'FL', 'id':11}, {'name':'FR', 'id':12},
          {'name':'KC', 'id':13}, {'name':'FLR', 'id':14},
          {'name':'L', 'id':15}, {'name':'ped', 'id':16},
          {'name':'rail', 'id':17}, {'name':'R', 'id':18},
          {'name':'school', 'id':19},
          {'name':'signal', 'id':20}, {'name':'stop', 'id':21},
          {'name':'xing', 'id':22}, {'name':'yield', 'id':23},
    
           # 9 helper classes
          {'name':'biker', 'id':24}, {'name':'car', 'id':25},
          {'name':'pedestrian', 'id':26}, {'name':'traffic_light', 'id':27},
          {'name':'truck', 'id':28}, {'name':'stop_sign', 'id':29},
          {'name':'yield_sign', 'id':30}, {'name':'school_sign', 'id':31},
          {'name':'ped_sign', 'id':32},
    
# 80 COCO classes
    {'name': 'person', 'id':33}, {'name': 'bicycle', 'id':34}, 
    {'name': 'car2', 'id':34},    # merge 2 'car' categories, replace this with dummy class
    {'name': 'motorbike', 'id':36},
    {'name': 'aeroplane', 'id':37}, {'name': 'bus', 'id':38}, {'name': 'train', 'id':39}, 
    {'name': 'truck2', 'id':40},    # merge truck and replace with dummy class
    {'name': 'boat', 'id':41}, {'name': 'traffic light', 'id':42}, {'name': 'fire hydrant', 'id':43}, {'name': 'stop sign', 'id':44}, 
    {'name': 'parking meter', 'id':45}, {'name': 'bench', 'id':46}, {'name': 'bird', 'id':47}, {'name': 'cat', 'id':48},
    {'name': 'dog', 'id':49}, {'name': 'horse', 'id':50}, {'name': 'sheep', 'id':51}, {'name': 'cow', 'id':52}, 
    {'name': 'elephant', 'id':53}, {'name': 'bear', 'id':54}, {'name': 'zebra', 'id':55}, {'name': 'giraffe', 'id':56},
    {'name': 'backpack', 'id':57}, {'name': 'umbrella', 'id':58}, {'name': 'handbag', 'id':59}, {'name': 'tie', 'id':60}, 
    {'name': 'suitcase', 'id':61}, {'name': 'frisbee', 'id':62}, {'name': 'skis', 'id':63}, {'name': 'snowboard', 'id':64},
    {'name': 'sports ball', 'id':65}, {'name': 'kite', 'id':66}, {'name': 'baseball bat', 'id':67}, {'name': 'baseball glove', 'id':68},
    {'name': 'skateboard', 'id':69}, {'name': 'surfboard', 'id':70}, {'name': 'tennis racket', 'id':71}, {'name': 'bottle', 'id':72},
    {'name': 'wine glass', 'id':73}, {'name': 'cup', 'id':74}, {'name': 'fork', 'id':75}, {'name': 'knife', 'id':76},
    {'name': 'spoon', 'id':77}, {'name': 'bowl', 'id':78}, {'name': 'banana', 'id':79}, {'name': 'apple', 'id':80},
    {'name': 'sandwich', 'id':81}, {'name': 'orange', 'id':82}, {'name': 'broccoli', 'id':83}, {'name': 'carrot', 'id':84},
    {'name': 'hot dog', 'id':85}, {'name': 'pizza', 'id':86}, {'name': 'donut', 'id':87}, {'name': 'cake', 'id':88},
    {'name': 'chair', 'id':89}, {'name': 'sofa', 'id':90}, {'name': 'pottedplant', 'id':91}, {'name': 'bed', 'id':92},
    {'name': 'diningtable', 'id':93}, {'name': 'toilet', 'id':94}, {'name': 'tvmonitor', 'id':95}, {'name': 'laptop', 'id':96},
    {'name': 'mouse', 'id':97}, {'name': 'remote', 'id':98}, {'name': 'keyboard', 'id':99}, {'name': 'cell phone', 'id':100},
    {'name': 'microwave', 'id':101}, {'name': 'oven', 'id':102}, {'name': 'toaster', 'id':103}, {'name': 'sink', 'id':104},
    {'name': 'refrigerator', 'id':105}, {'name': 'book', 'id':106}, {'name': 'clock', 'id':107}, {'name': 'vase', 'id':108},
    {'name': 'scissors', 'id':109}, {'name': 'teddy bear', 'id':110}, {'name': 'hair drier', 'id':111}, {'name': 'toothbrush', 'id':112} ]

#### Separate NMS and confidence filter functions

In [5]:
########  NMS block, to remove redundant bboxes  ########
def nms(rects, thd=0.5):    # rects here probably refer to detections
    """
    Filter rectangles
    rects is array of oblects ([x1,y1,x2,y2], confidence, class)
    thd - intersection threshold (intersection divides min square of rectange)    # same as IoU?
    """
    out = []
    remove = [False] * len(rects)
    for i in range(0, len(rects) - 1):
        if remove[i]:
            continue
        inter = [0.0] * len(rects)
        for j in range(i, len(rects)):
            if remove[j]:
                continue
            inter[j] = intersection(rects[i][0], rects[j][0]) / min(square(rects[i][0]), square(rects[j][0]))
        max_prob = 0.0
        max_idx = 0
        for k in range(i, len(rects)):
            if inter[k] >= thd:
                if rects[k][1] > max_prob:
                    max_prob = rects[k][1]
                    max_idx = k
        for k in range(i, len(rects)):
            if (inter[k] >= thd) & (k != max_idx):
                remove[k] = True
    for k in range(0, len(rects)):
        if not remove[k]:
            out.append(rects[k])
    boxes = [box[0] for box in out]
    scores = [score[1] for score in out]
    classes = [cls[2] for cls in out]
    return boxes, scores, classes

def intersection(rect1, rect2):
    """
    Calculates square of intersection of two rectangles
    rect: list with coords of top-right and left-boom corners [x1,y1,x2,y2]
    return: square of intersection
    """
    x_overlap = max(0, min(rect1[2], rect2[2]) - max(rect1[0], rect2[0]));
    y_overlap = max(0, min(rect1[3], rect2[3]) - max(rect1[1], rect2[1]));
    overlapArea = x_overlap * y_overlap;
    return overlapArea

def square(rect):
    """ Calculates square of rectangle """
    return abs(rect[2] - rect[0]) * abs(rect[3] - rect[1])

########  Confidence Filter block  ########
def predictions_above_confidence(boxes_nms, scores_nms, labels_nms, conf_thd):
    boxes_final, scores_final, labels_final = [], [], []     # create empty placeholder lists
    for pred in zip(boxes_nms, scores_nms, labels_nms):
        b, s, l = pred    # unpack predicted boxes, score, label
        if s >= conf_thd:    # select boxes if confidence > threshold
            boxes_final.append(b)
            scores_final.append(s)
            labels_final.append(l)
    return boxes_final, scores_final, labels_final


#### Write ensemble/NMS output to .txt

In [6]:
#### parse ensemble master lists to this fn, to create new txt files

def write_predictions_txt(boxes_list, scores_list, labels_list, out_path, file_name):
    # create and open a txt file
    with open(os.path.join(out_path, file_name), 'w') as f:    # txt needs to have same name as input image/annotation
        # loop thru all predicted boxes for this image
        for box, score, label in zip(boxes_list, scores_list, labels_list):
            x1, y1, x2, y2 = box    # This is the format of RODM
            # convert into yolo [x,y,w,h] in RELATIVE numbers
            cx = (x1 + x2) / 2    # x-center
            cy = (y1 + y2) / 2    # y-center
            w = x2 - x1    # width
            h = y2 - y1    # height
            line = str(int(label))+' '+f'{score:.5f}'+' '+str(cx)+' '+str(cy)+' '+str(w)+' '+str(h)+'\n'
            f.write(line)

#### Convert input formats to fit ensemble_boxes

In [13]:
# read each file parsed into this fn
# and convert from RODM txt format [x,y,w,h] to EB format[x1,y1,x2,y2]

def load_pred(file):
    # instantiate placeholder vars
    boxes_list, scores_list, labels_list = [], [], []
    
    with open(file, 'r') as f:
        for line in f.readlines():    # read each line
            # extract info for each predicted box
            line = line.split()    # make line elements iterable
            labels_list.append(int(line[0]))    # class id should be int
            scores_list.append(float(line[1]))    # confidence score shoulbe be float

            # convert xywh to [x1, y1, x2, y2]
            x, y, w, h = float(line[2]), float(line[3]), float(line[4]), float(line[5])
            x1 = x - w/2
            y1 = y - h/2
            x2 = x + w/2
            y2 = y + h/2
            boxes_list.append([x1, y1, x2, y2])
    
    return boxes_list, scores_list, labels_list


#### EB WBF Example

In [8]:
def example_wbf_2_models(image_path, iou_thr=0.55, draw_image=True):
    """
    This example shows how to ensemble boxes from 2 models using WBF method    
    :return: 
    """
    boxes_list = [
        [
            [0.00, 0.51, 1.00, 0.91],
            [0.10, 0.31, 0.71, 0.61],
            [0.01, 0.32, 0.83, 0.93],
            [0.02, 0.53, 0.11, 0.94],
            [0.03, 0.24, 0.12, 0.35],
        ],
        [
            [0.04, 0.56, 0.84, 0.92],
            [0.12, 0.33, 0.72, 0.64],
            [0.38, 0.66, 0.79, 0.95],
            [0.08, 0.49, 0.21, 0.89],
        ],
    ]
    scores_list = [ [0.9, 0.8, 0.2, 0.4, 0.7],
                    [0.5, 0.8, 0.7, 0.3] ]
    labels_list = [ [10, 28, 6, 1, 2],
                    [5, 7, 9, 11] ]
    weights = [2, 1]
    
    # show image before WBF
    image_path = image_path
    if draw_image:
        # added image_path
        show_boxes(boxes_list, scores_list, labels_list, image_path)
    
    # apply WBF
    boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=0.0)
    
    # show image after WBF
    if draw_image:
        # added image_path
        show_boxes([boxes], [scores], [labels.astype(np.int32)], image_path)

    # print bboxes after WBF
    print(len(boxes))
    print(boxes)

#### Optional: apply NMS and confidence filter on model predictions

In [17]:
#### Note: It is better to run NMS and confidence filter when running inference.

""" 
Apply NMS and confidence filter to default 100 bbox predictions for SSD.
And write processed predictions to txt files.
So we have 2 sets of predictions for SSD.
"""

# where SSD default 100 bbox predictions are stored
pred_path = r'D:\DL_data\Accuracy\SSD_predictions_100bboxes'

# where do you want to store processed predictions
out_path = r'D:\DL_data\Accuracy\SSD_nms_conf'

# set confidence threshold
conf_thd = 0.3    # for SSD, 0.3 is good in general

# loop thru all predictions
for file in os.listdir(pred_path):
    image_name = file.split('.')[0]    # extract file name without file extension
    
    # open prediction txt file
    with open( os.path.join(pred_path, file), 'r') as f:
        boxes_list, scores_list, labels_list = [], [], []    # placeholders
        
        for line in f.readlines():    # read each line
            line = line.split()    # make line elements iterable
            label = int(line[0])
            score = float(line[1])
            x, y, w, h = float(line[2]), float(line[3]), float(line[4]), float(line[5])
            
            boxes_list.append([x, y, w, h])
            scores_list.append(score)
            labels_list.append(label)
            
    # generate rectangles for nms fn
    rectangles = [(b, s, l) for (b, s, l) in zip(boxes_list, scores_list, labels_list)]

    # apply NMS to predictions
    boxes_nms, scores_nms, labels_nms = nms(rectangles)    # dafault IoU threshold = 0.5
    
    # apply confidence filter 
    boxes_final, scores_final, labels_final = predictions_above_confidence(boxes_nms, scores_nms, labels_nms, conf_thd)

    # write processed predictions to new txt files
    write_predictions_txt(boxes_final, scores_final, labels_final, image_name, out_path)

In [8]:
#### confidence filter

def predictions_above_confidence(boxes_nms, scores_nms, labels_nms, conf_thd):
    boxes_final, scores_final, labels_final = [], [], []     # create empty placeholder lists
    for pred in zip(boxes_nms, scores_nms, labels_nms):
        b, s, l = pred    # unpack predicted boxes, score, label
        if s >= conf_thd:    # select boxes if confidence > threshold
            boxes_final.append(b)
            scores_final.append(s)
            labels_final.append(l)
    return boxes_final, scores_final, labels_final


## Stage 1 Ensemble

In [38]:
#### Note: Stage 1 code needs to update paths, because folder structure has changed

In [7]:
"""
Ensemble pseudo code

for image in folder:
    create master lists
    
    for model in models:
        get predictions
        append to master lists
        
    do ensemble 
    write output to new txt files

# below may be done thru another program/notebook
calculate mAP
show some images
"""

'\nEnsemble pseudo code\n\nfor image in folder:\n    create master lists\n    \n    for model in models:\n        get predictions\n        append to master lists\n        \n    do ensemble \n    write output to new txt files\n\n# below may be done thru another program/notebook\ncalculate mAP\nshow some images\n'

In [8]:
#### locate model predictions txt files for each model

# custom YOLOv4
path_YOLO4 = r'D:\DL_data\Accuracy\YOLO_test_output'

# custom EfficientDet
path_ED = r'D:\DL_data\Accuracy\Eff_test_results'

# custom SSD mobilenet 640
path_SSD = r'D:\DL_data\Accuracy\SSD_nms_conf'    # with NMS and confidence filter applied

# custom CenterNet Hourglass 512
path_CN = r'D:\DL_data\Accuracy\CenterNet_nms_conf'

In [9]:
# choose where to store ensemble output txt files
out_path = r'D:\DL_data\Accuracy\Ensemble_Output'

In [35]:
#### some params for Weighted Boxes Fusion should you want to tweek

# weights for each model:
weights = [2, 1.5, 1, 1.5]    # here we assume the order is [Yolo4, EfficientDet, SSD, CenterNet]

# IoU threshold
iou_thr = 0.5    # use 0.5 for COCO mAP

# confidence threshold, for confidence filter
conf_thd = 0.25

In [36]:
""" Run this cell to generate ensemble results and write to txt files """

# generate a list of all images
# since all models have predictions on the same test dataset, we can get this list from any 1 model
all_img_preds = os.listdir(path_YOLO4)    # get list of file names without full path

# gather paths for all 4 models
model_paths = [path_YOLO4, path_ED, path_SSD, path_CN]

# work on 1 image at a time
for img_pred in all_img_preds:
    # instantiate master lists
    boxes_master = []
    scores_master = []
    labels_master = []
    # note: img_pred is something like 'A101.txt'
    
    # for this image, gather predictions from all 4 models, model by model
    for model_path in model_paths:
        # instantiate placeholders for this model
        boxes_model = []
        scores_model = []
        labels_model = []
        
        # locate the prediction txt file
        file = os.path.join(model_path, img_pred)
        # read prediction file
        with open(file, 'r') as f:
            # read each line in prediction file
            for line in f.readlines():
                line = line.split()    # make line elements iterable
                labels_model.append(int(line[0]))    # class id should be int
                scores_model.append(float(line[1]))    # confidence score shoulbe be float
                
                # convert xywh to [x1, y1, x2, y2]
                x, y, w, h = float(line[2]), float(line[3]), float(line[4]), float(line[5])
                x1 = x - w/2
                y1 = y - h/2
                x2 = x + w/2
                y2 = y + h/2
                boxes_model.append([x1, y1, x2, y2])
                
        # now gather all 4 model predictions for this image 
        boxes_master.append(boxes_model)
        scores_master.append(scores_model)
        labels_master.append(labels_model)

    # apply ensemble to all 4 model predictions for this image
    boxes_ensemble, scores_ensemble, labels_ensemble = weighted_boxes_fusion(boxes_master, scores_master, labels_master, weights=weights, iou_thr=iou_thr, skip_box_thr=0.0)
    
#     # apply NMS to remove redundant boxes
#     boxes_e_nms, scores_e_nms, labels_e_nms = nms_method([boxes_ensemble], [scores_ensemble], [labels_ensemble], method=3, weights=weights, iou_thr=iou_thr, sigma=0.5, thresh=0.001)

    # apply confidence filter
    boxes_e_c, scores_e_c, labels_e_c = predictions_above_confidence(boxes_ensemble, scores_ensemble, labels_ensemble, conf_thd)

    # write ensemble results to txt file, we use the same file name as predictions, just store to different folder        
    write_predictions_txt(boxes_e_c, scores_e_c, labels_e_c, out_path, img_pred)

## Stage 2 Ensemble

In [37]:
"""
As mentioned, we cannot calculate mAP for Stage 2, because our test data is not annotated for all 80 COCO classes.
However, we can show prediction images for Stage 2 ensemble model.

Note: we are re-using the names of variables. 
So DO NOT skip/jump around when executing Stage 1 and Stage 2 code.

"""

'\nAs mentioned, we cannot calculate mAP for Stage 2, because our test data is not annotated for all 80 COCO classes.\nHowever, we can show prediction images for Stage 2 ensemble model.\n'

In [9]:
# gather a new tiny test dataset (let's say 5 images)
# and run all 4 models
# ensemble the predictions (stage 1)
# then run stock YOLOv4 on new tiny test dataset
# ensemble this results and the stage 1 results
# visualize stage 2 results

#### set up paths for stage 2

In [63]:
# stage 2 test data location
s2_img_path = r'D:\DL_data\Accuracy\Stage_2_Ensemble\Test_images'

In [71]:
#### locate model predictions txt files for each model

# CUSTOM YOLOv4
path_YOLO4 = r'D:\DL_data\Accuracy\Stage_2_Ensemble\custom_yolo4'

# custom EfficientDet
path_ED = r'D:\DL_data\Accuracy\Stage_2_Ensemble\efficientdet'

# custom SSD mobilenet 640
path_SSD = r'D:\DL_data\Accuracy\Stage_2_Ensemble\ssd'    # with NMS and confidence filter applied

# custom CenterNet Hourglass 512
path_CN = r'D:\DL_data\Accuracy\Stage_2_Ensemble\centernet'

# STOCK YOLOv4
path_stock_YOLO4 = r'D:\DL_data\Accuracy\Stage_2_Ensemble\stock_yolo4'

# stage 1 ensemble output, for this new tiny test dataset
stage1_output = r'D:\DL_data\Accuracy\Stage_2_Ensemble\stage1_output'

In [65]:
# choose where to store stage 2 output txt files
stage2_output = r'D:\DL_data\Accuracy\Stage_2_Ensemble\stage2_output'

#### visualize some model predictions before stage 1 ensemble

In [37]:
# show some stage prelim predictions

# show SSD prediction on this image
file_name = '9663077996_31dd326967_z'

boxes_list, scores_list, labels_list = load_pred(os.path.join(path_ED,file_name+'.txt'))

show_boxes([boxes_list], [scores_list], [labels_list], os.path.join(s2_img_path,file_name+'.jpg'))

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\DL_data\\Accuracy\\Stage 2 Ensemble\\efficientdet\\9663077996_31dd326967_z.txt'

#### do stage 2 ensemble for new tiny test dataset

In [39]:
#### now that we have predictions of new tiny test dataset from all 4 custom models + stock YOLOv4
## we can proceed to do stage 1 ensemble for this test dataset

In [44]:
#### some params for Weighted Boxes Fusion should you want to tweek

# weights for each model:
weights = [2, 1.5, 1, 1.5]    # here we assume the order is [Yolo4, EfficientDet, SSD, CenterNet]

# IoU threshold
iou_thr = 0.5    # use 0.5 for COCO mAP

# confidence threshold, for confidence filter
conf_thd = 0.25

In [52]:
""" Run this cell to generate ensemble results and write to txt files """

# generate a list of all images
# since all models have predictions on the same test dataset, we can get this list from any 1 model
all_img_preds = os.listdir(path_YOLO4)    # get list of file names without full path

# gather paths for all 4 models
model_paths = [path_YOLO4, path_ED, path_SSD, path_CN]

# work on 1 image at a time
for img_pred in all_img_preds:
    # instantiate master lists
    boxes_master = []
    scores_master = []
    labels_master = []
    # note: img_pred is something like 'A101.txt'
    
    # for this image, gather predictions from all 4 models, model by model
    for model_path in model_paths:
        # instantiate placeholders for this model
        boxes_model = []
        scores_model = []
        labels_model = []
        
        # locate the prediction txt file
        file = os.path.join(model_path, img_pred)
        # read prediction file
        with open(file, 'r') as f:
            # read each line in prediction file
            for line in f.readlines():
                line = line.split()    # make line elements iterable
                labels_model.append(int(line[0]))    # class id should be int
                scores_model.append(float(line[1]))    # confidence score shoulbe be float
                
                # convert xywh to [x1, y1, x2, y2]
                x, y, w, h = float(line[2]), float(line[3]), float(line[4]), float(line[5])
                x1 = x - w/2
                y1 = y - h/2
                x2 = x + w/2
                y2 = y + h/2
                boxes_model.append([x1, y1, x2, y2])
                
        # now gather all 4 model predictions for this image 
        boxes_master.append(boxes_model)
        scores_master.append(scores_model)
        labels_master.append(labels_model)

    # apply ensemble to all 4 model predictions for this image
    boxes_ensemble, scores_ensemble, labels_ensemble = weighted_boxes_fusion(boxes_master, scores_master, labels_master, weights=weights, iou_thr=iou_thr, skip_box_thr=0.0)
    
#     # apply NMS to remove redundant boxes
#     boxes_e_nms, scores_e_nms, labels_e_nms = nms_method([boxes_ensemble], [scores_ensemble], [labels_ensemble], method=3, weights=weights, iou_thr=iou_thr, sigma=0.5, thresh=0.001)

    # apply confidence filter
    boxes_e_c, scores_e_c, labels_e_c = predictions_above_confidence(boxes_ensemble, scores_ensemble, labels_ensemble, conf_thd)

    # write ensemble results to txt file, we use the same file name as predictions, just store to different folder        
    write_predictions_txt(boxes_e_c, scores_e_c, labels_e_c, stage1_output, img_pred)

#### visualize stage 1 ensemble

In [53]:
# show some stage 1 ensemble predictions

# loop thru all test images
eight_imgs = os.listdir(s2_img_path)
for img in eight_imgs:
#     print(img)
    file_name = img.split('.')[0]
    print(file_name)
    
    # convert RODM format to EB format
    boxes_list, scores_list, labels_list = load_pred(os.path.join(stage1_output,file_name+'.txt'))

    # visualize ensemble predictions for each image
    show_boxes([boxes_list], [scores_list], [labels_list], os.path.join(s2_img_path,file_name+'.jpg'))

2058486321_8c8fbcbcd7_z
6828176341_3a23aa4623_z
8288873861_a24b6b421a_z
8302503786_8820161815_z
9663077996_31dd326967_z
A858
B193
B365


#### run stage 2 ensemble

In [54]:
#### now that we have stage 1 ensemble from 4 custom models, 
## we can do stage 2 ensemble using stage 1 output and stock YOLOv4 output

In [57]:
# Update weights for stage 1 output and stock yolo4:
weights = [1, 1]    # here we assume the order is [Stage 1 ensemble, Stock YOLOv4]

In [86]:
""" Run this cell to run Stage 2 ensemble and write results to txt files """

# generate a list of all images
# since all models have predictions on the same test dataset, we can get this list from any 1 model
all_img_preds = os.listdir(path_YOLO4)    # get list of file names without full path

# gather paths for all 4 models
model_paths = [stage1_output, path_stock_YOLO4]

# work on 1 image at a time
for img_pred in all_img_preds:
    # instantiate master lists
    boxes_master = []
    scores_master = []
    labels_master = []
    # note: img_pred is something like 'A101.txt'
    
    # for this image, gather predictions from all 4 models, model by model
    for model_path in model_paths:
        # instantiate placeholders for this model
        boxes_model = []
        scores_model = []
        labels_model = []
        
        # locate the prediction txt file
        file = os.path.join(model_path, img_pred)
        # read prediction file
        with open(file, 'r') as f:
            # read each line in prediction file
            for line in f.readlines():
                line = line.split()    # make line elements iterable
                labels_model.append(int(line[0]))    # class id should be int
                scores_model.append(float(line[1]))    # confidence score shoulbe be float
                
                # convert xywh to [x1, y1, x2, y2]
                x, y, w, h = float(line[2]), float(line[3]), float(line[4]), float(line[5])
                x1 = x - w/2
                y1 = y - h/2
                x2 = x + w/2
                y2 = y + h/2
                boxes_model.append([x1, y1, x2, y2])
                
        # now gather all 4 model predictions for this image 
        boxes_master.append(boxes_model)
        scores_master.append(scores_model)
        labels_master.append(labels_model)

    # apply ensemble to all 4 model predictions for this image
    boxes_ensemble, scores_ensemble, labels_ensemble = weighted_boxes_fusion(boxes_master, scores_master, labels_master, weights=weights, iou_thr=iou_thr, skip_box_thr=0.0)
    
#     # apply NMS to remove redundant boxes
#     boxes_e_nms, scores_e_nms, labels_e_nms = nms_method([boxes_ensemble], [scores_ensemble], [labels_ensemble], method=3, weights=weights, iou_thr=iou_thr, sigma=0.5, thresh=0.001)

    # apply confidence filter
    boxes_e_c, scores_e_c, labels_e_c = predictions_above_confidence(boxes_ensemble, scores_ensemble, labels_ensemble, conf_thd)

    # write ensemble results to txt file, we use the same file name as predictions, just store to different folder        
    write_predictions_txt(boxes_e_c, scores_e_c, labels_e_c, stage2_output, img_pred)

C:\Users\168\Anaconda3\envs\t4\lib\site-packages\ensemble_boxes\ensemble_boxes_wbf.py:85: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
C:\Users\168\Anaconda3\envs\t4\lib\site-packages\ensemble_boxes\ensemble_boxes_wbf.py:73: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')


#### visualize stage 2 ensemble

In [85]:
# show some stock YOLOv4 predicitons on new tiny test dataset
## new window will pop open, press any key to destroy and let new image pop up

# loop thru all test images
eight_imgs = os.listdir(s2_img_path)
for img in eight_imgs:
    file_name = img.split('.')[0]
    print(file_name)
    
    # convert RODM format to EB format
    boxes_list, scores_list, labels_list = load_pred(os.path.join(path_stock_YOLO4,file_name+'.txt'))

    # visualize ensemble predictions for each image
    show_boxes([boxes_list], [scores_list], [labels_list], os.path.join(s2_img_path,file_name+'.jpg'))

2058486321_8c8fbcbcd7_z
6828176341_3a23aa4623_z
8288873861_a24b6b421a_z
8302503786_8820161815_z
9663077996_31dd326967_z
A858
B193
B365


In [98]:
# show stage 2 ensemble on new tiny test dataset
## new window will pop open, press any key to destroy and let new image pop up

# loop thru all test images
eight_imgs = os.listdir(s2_img_path)
for img in eight_imgs:
    file_name = img.split('.')[0]
    print(file_name)
    
    # convert RODM format to EB format
    boxes_list, scores_list, labels_list = load_pred(os.path.join(stage2_output,file_name+'.txt'))

    # visualize ensemble predictions for each image
    show_boxes([boxes_list], [scores_list], [labels_list], os.path.join(s2_img_path,file_name+'.jpg'))

2058486321_8c8fbcbcd7_z
6828176341_3a23aa4623_z
8288873861_a24b6b421a_z
8302503786_8820161815_z
9663077996_31dd326967_z
A858
B193
B365
